In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('../data/train_featureV1.csv')
test = pd.read_csv('../data/test_featureV1.csv')

In [3]:
train.head()

,uid,label,voice_opp_num_unique_count,voice_opp_num_count,voice_opp_head_unique_count,voice_opp_len_3,voice_opp_len_5,voice_opp_len_6,voice_opp_len_7,voice_opp_len_8,...,visit_dura_x_y.10,visit_dura_y_y.10,visit_dura_x_y.11,visit_dura_y_y.11,visit_dura_x_y.12,visit_dura_y_y.12,visit_dura_x_y.13,visit_dura_y_y.13,visit_dura_x_y.14,visit_dura_y_y.14
0,u0001,0,22.0,79.0,17.0,0.0,3.0,0.0,0.0,0.0,...,47692.0,13190.0,17314.0,14337.0,34536.0,17084.0,4806.0,9402.0,3378084.0,8907.0
1,u0002,0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,u0003,0,15.0,21.0,10.0,0.0,1.0,0.0,0.0,2.0,...,25104.0,17164.0,18219.0,38717.0,1039108.0,440294.0,175357.0,2184.0,452927.0,2281.0
3,u0004,0,77.0,254.0,31.0,0.0,1.0,0.0,0.0,12.0,...,145443.0,22237.0,10694.0,513625.0,110245.0,440247.0,607104.0,15360.0,1971461.0,5034.0
4,u0005,0,55.0,401.0,28.0,0.0,4.0,0.0,0.0,0.0,...,2343340.0,0.0,0.0,4844.0,82539.0,9950.0,0.0,1958.0,174052.0,160872.0


In [4]:
dtrain = lgb.Dataset(train.drop(['uid','label'],axis=1),label=train.label)
dtest = lgb.Dataset(test.drop(['uid'],axis=1))

In [5]:
def evalMetric(preds,dtrain):
    
    label = dtrain.get_label()
    
    
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    
    auc = metrics.roc_auc_score(pre.label,pre.preds)

    pre.preds=pre.preds.map(lambda x: 1 if x>=0.38 else 0)

    f1 = metrics.f1_score(pre.label,pre.preds)
    
    
    res = 0.6*auc +0.4*f1
    
    return 'res',res,True
    

    

## lgb

In [6]:
lgb_params =  {
    'seed':0,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'is_training_metric': False,
    'learning_rate': 0.02,
    'verbosity':-1,
    'colsample_bytree':0.7,
}

lgb_params1 = lgb_params.copy()
lgb_params1['seed'] = 1

lgb_params2 = lgb_params.copy()
lgb_params2['seed'] = 2

lgb_params3 = lgb_params.copy()
lgb_params3['seed'] = 3

lgb_params4 = lgb_params.copy()
lgb_params4['seed'] = 4

lgb_params5 = lgb_params.copy()
lgb_params5['seed'] = 5

lgb_params6 = lgb_params.copy()
lgb_params6['seed'] = 6

lgb_params7 = lgb_params.copy()
lgb_params7['seed'] = 7

lgb_params8 = lgb_params.copy()
lgb_params8['seed'] = 8

lgb_params9 = lgb_params.copy()
lgb_params9['seed'] = 9



In [7]:
lgb_params1

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.7,
 'is_training_metric': False,
 'learning_rate': 0.02,
 'objective': 'binary',
 'seed': 1,
 'verbosity': -1}

### 本地CV

In [8]:
lgb.cv(lgb_params,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.65207 + 0.00427134
[4]	cv_agg's res: 0.660981 + 0.00128633
[6]	cv_agg's res: 0.665121 + 0.00410356
[8]	cv_agg's res: 0.677533 + 0.00498726
[10]	cv_agg's res: 0.679002 + 0.0049864
[12]	cv_agg's res: 0.679177 + 0.00570212
[14]	cv_agg's res: 0.680107 + 0.00460016
[16]	cv_agg's res: 0.788216 + 0.0036326
[18]	cv_agg's res: 0.812539 + 0.013179
[20]	cv_agg's res: 0.82229 + 0.013217
[22]	cv_agg's res: 0.828693 + 0.0117303
[24]	cv_agg's res: 0.834183 + 0.0111809
[26]	cv_agg's res: 0.836843 + 0.0126654
[28]	cv_agg's res: 0.840547 + 0.0120337
[30]	cv_agg's res: 0.841203 + 0.0114359
[32]	cv_agg's res: 0.844804 + 0.0109931
[34]	cv_agg's res: 0.845313 + 0.0109152
[36]	cv_agg's res: 0.846973 + 0.010346
[38]	cv_agg's res: 0.847199 + 0.0112122
[40]	cv_agg's res: 0.847139 + 0.0104083
[42]	cv_agg's res: 0.847622 + 0.0105826
[44]	cv_agg's res: 0.849876 + 0.0106004
[46]	cv_agg's res: 0.85039 + 0.00966496
[48]	cv_agg's res: 0.850673 + 0.00968642
[50]	cv_agg's res: 0.850777 + 0.0103392
[5

{'res-mean': [0.6338216368208713,
  0.6520695194392236,
  0.6563075126723135,
  0.6609814407182888,
  0.6627930357904473,
  0.6651214629890415,
  0.675722348463891,
  0.6775329878001526,
  0.6773237885654554,
  0.6790019163870094,
  0.6792888037047833,
  0.6791765763188975,
  0.679244023532965,
  0.6801070214546469,
  0.758167819881734,
  0.7882156598303647,
  0.8038856503418549,
  0.8125392700381742,
  0.818074744608737,
  0.8222899999187628,
  0.8257808505559501,
  0.8286932503032922,
  0.8309473342601068,
  0.8341829124961295,
  0.834507199481172,
  0.8368426380318846,
  0.8384709611367837,
  0.8405470644427894,
  0.8408248774937551,
  0.8412030766187484,
  0.8430238490193144,
  0.8448038194710513,
  0.8445570234849239,
  0.8453127704803315,
  0.8458154634512293,
  0.8469727897676522,
  0.8469824754124842,
  0.8471991892354516,
  0.8475037152405077,
  0.8471391619928882,
  0.8473536348053458,
  0.8476221413489028,
  0.849459181770332,
  0.849875971534094,
  0.8493659532734368,
  0.8

## 训练

In [9]:
model =lgb.train(lgb_params,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=226,valid_sets=[dtrain])

[100]	training's res: 0.941134
[200]	training's res: 0.976264


In [10]:
lgb.cv(lgb_params1,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.652562 + 0.00704263
[4]	cv_agg's res: 0.657798 + 0.00331297
[6]	cv_agg's res: 0.660335 + 0.00341213
[8]	cv_agg's res: 0.665255 + 0.00895263
[10]	cv_agg's res: 0.674621 + 0.00599156
[12]	cv_agg's res: 0.678208 + 0.00406829
[14]	cv_agg's res: 0.678778 + 0.00482094
[16]	cv_agg's res: 0.78318 + 0.00600354
[18]	cv_agg's res: 0.811985 + 0.00817751
[20]	cv_agg's res: 0.82314 + 0.0101535
[22]	cv_agg's res: 0.83502 + 0.00878919
[24]	cv_agg's res: 0.839318 + 0.00862676
[26]	cv_agg's res: 0.84314 + 0.0126161
[28]	cv_agg's res: 0.844844 + 0.0128176
[30]	cv_agg's res: 0.847423 + 0.0138185
[32]	cv_agg's res: 0.849856 + 0.0132884
[34]	cv_agg's res: 0.850354 + 0.0106697
[36]	cv_agg's res: 0.851238 + 0.00963629
[38]	cv_agg's res: 0.850908 + 0.0104047
[40]	cv_agg's res: 0.851998 + 0.0107573
[42]	cv_agg's res: 0.852814 + 0.0115657
[44]	cv_agg's res: 0.85334 + 0.0117979
[46]	cv_agg's res: 0.853972 + 0.0114282
[48]	cv_agg's res: 0.854324 + 0.011113
[50]	cv_agg's res: 0.855051 + 0.011082

{'res-mean': [0.6448647606099125,
  0.652562232176145,
  0.6562220523178595,
  0.6577979249668544,
  0.6585325986510574,
  0.6603347388282738,
  0.6636646294416763,
  0.6652549589795563,
  0.6716206227483119,
  0.6746211934408152,
  0.676887546003469,
  0.6782078510535174,
  0.6787225346485268,
  0.6787782208129913,
  0.7505715065681527,
  0.78318047222905,
  0.7998964612956937,
  0.8119849260080021,
  0.8179556401794349,
  0.8231397830410017,
  0.8289235709201493,
  0.83501999571574,
  0.837901995251164,
  0.8393181528518462,
  0.8411807813486956,
  0.8431398355015748,
  0.8442776693236995,
  0.8448436638723379,
  0.8466568224229082,
  0.8474231885575683,
  0.8486083682867097,
  0.8498560564977075,
  0.8504204881538527,
  0.8503535428594597,
  0.8507999362909898,
  0.8512376982768467,
  0.8507372980310587,
  0.8509082809637473,
  0.8513215708664484,
  0.8519977711256361,
  0.8509849581800589,
  0.8528136647143739,
  0.8530581277929633,
  0.8533404679695759,
  0.8542136314115633,
  0.8

In [11]:
model1 =lgb.train(lgb_params1,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=222,valid_sets=[dtrain])

[100]	training's res: 0.942113
[200]	training's res: 0.977627


In [12]:
lgb.cv(lgb_params2,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.657837 + 0.00826115
[4]	cv_agg's res: 0.668987 + 0.003194
[6]	cv_agg's res: 0.674212 + 0.00333253
[8]	cv_agg's res: 0.675044 + 0.00356708
[10]	cv_agg's res: 0.677815 + 0.0021239
[12]	cv_agg's res: 0.68003 + 0.00392174
[14]	cv_agg's res: 0.689938 + 0.00992971
[16]	cv_agg's res: 0.786836 + 0.00614869
[18]	cv_agg's res: 0.813582 + 0.0119762
[20]	cv_agg's res: 0.824707 + 0.0133289
[22]	cv_agg's res: 0.832945 + 0.0127818
[24]	cv_agg's res: 0.838537 + 0.0150366
[26]	cv_agg's res: 0.8427 + 0.0149544
[28]	cv_agg's res: 0.846466 + 0.0139973
[30]	cv_agg's res: 0.846102 + 0.0144665
[32]	cv_agg's res: 0.847468 + 0.0132102
[34]	cv_agg's res: 0.847462 + 0.0116163
[36]	cv_agg's res: 0.848459 + 0.0128324
[38]	cv_agg's res: 0.850146 + 0.0115573
[40]	cv_agg's res: 0.849565 + 0.0109951
[42]	cv_agg's res: 0.849747 + 0.0115146
[44]	cv_agg's res: 0.850743 + 0.0127434
[46]	cv_agg's res: 0.851871 + 0.0124058
[48]	cv_agg's res: 0.853076 + 0.0125371
[50]	cv_agg's res: 0.854015 + 0.0125176
[5

[406]	cv_agg's res: 0.866057 + 0.0114127
[408]	cv_agg's res: 0.86608 + 0.0114114
[410]	cv_agg's res: 0.866203 + 0.0119207
[412]	cv_agg's res: 0.866519 + 0.0115907
[414]	cv_agg's res: 0.866218 + 0.0117296
[416]	cv_agg's res: 0.866131 + 0.011267


{'res-mean': [0.6490151213939658,
  0.6578374282412405,
  0.6697605218480208,
  0.6689873598902366,
  0.6717913142519731,
  0.6742118726773546,
  0.674536810386723,
  0.6750436429879919,
  0.6754002954560719,
  0.677814772623552,
  0.6777830046692302,
  0.6800303837143726,
  0.6809254332380578,
  0.6899380902877409,
  0.7612418486931568,
  0.7868358133244898,
  0.8056220892788164,
  0.8135819050704485,
  0.820222382105079,
  0.8247068624744484,
  0.8288038129063562,
  0.8329453104079715,
  0.8350713478963904,
  0.8385366334638807,
  0.8406292994880326,
  0.8427002276209015,
  0.8433712393098457,
  0.8464663010367203,
  0.8449437514549593,
  0.8461022937753034,
  0.8471119285463152,
  0.8474677337212224,
  0.8475188762382282,
  0.8474615449241863,
  0.8470968309610966,
  0.8484586289133317,
  0.8486913219569416,
  0.8501463254149696,
  0.8498946196726692,
  0.8495646559139081,
  0.8500988813407796,
  0.8497474318817423,
  0.8510200734112484,
  0.8507425117858948,
  0.8517587062652222,
 

In [14]:
model2 =lgb.train(lgb_params3,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=316,valid_sets=[dtrain])

[100]	training's binary_logloss: 0.197503	training's res: 0.942454
[200]	training's binary_logloss: 0.103487	training's res: 0.978334
[300]	training's binary_logloss: 0.0627248	training's res: 0.995345


In [15]:
lgb.cv(lgb_params3,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.649995 + 0.00417206
[4]	cv_agg's res: 0.661817 + 0.00280062
[6]	cv_agg's res: 0.663766 + 0.00170156
[8]	cv_agg's res: 0.667472 + 0.00461855
[10]	cv_agg's res: 0.676743 + 0.00419643
[12]	cv_agg's res: 0.676513 + 0.00425737
[14]	cv_agg's res: 0.6795 + 0.00423397
[16]	cv_agg's res: 0.784076 + 0.00743825
[18]	cv_agg's res: 0.810752 + 0.0103277
[20]	cv_agg's res: 0.823754 + 0.00946464
[22]	cv_agg's res: 0.830975 + 0.00907134
[24]	cv_agg's res: 0.838035 + 0.011496
[26]	cv_agg's res: 0.839292 + 0.0113769
[28]	cv_agg's res: 0.840895 + 0.0117459
[30]	cv_agg's res: 0.840256 + 0.0115019
[32]	cv_agg's res: 0.842742 + 0.00953766
[34]	cv_agg's res: 0.845231 + 0.00961169
[36]	cv_agg's res: 0.844712 + 0.00879526
[38]	cv_agg's res: 0.847436 + 0.00973964
[40]	cv_agg's res: 0.847917 + 0.0102028
[42]	cv_agg's res: 0.849788 + 0.00969474
[44]	cv_agg's res: 0.849766 + 0.00971712
[46]	cv_agg's res: 0.850728 + 0.0100824
[48]	cv_agg's res: 0.850712 + 0.00966262
[50]	cv_agg's res: 0.850862 + 

[406]	cv_agg's res: 0.865086 + 0.011648
[408]	cv_agg's res: 0.865059 + 0.0110572
[410]	cv_agg's res: 0.864616 + 0.011659
[412]	cv_agg's res: 0.864741 + 0.0115613
[414]	cv_agg's res: 0.864749 + 0.0115439
[416]	cv_agg's res: 0.864048 + 0.0116869
[418]	cv_agg's res: 0.864512 + 0.0120309
[420]	cv_agg's res: 0.864851 + 0.0118099
[422]	cv_agg's res: 0.864724 + 0.0119427
[424]	cv_agg's res: 0.864609 + 0.0120317
[426]	cv_agg's res: 0.864937 + 0.0114437
[428]	cv_agg's res: 0.865381 + 0.0118704
[430]	cv_agg's res: 0.8654 + 0.011894
[432]	cv_agg's res: 0.865424 + 0.0118941
[434]	cv_agg's res: 0.86461 + 0.0117559
[436]	cv_agg's res: 0.864051 + 0.0120105
[438]	cv_agg's res: 0.864946 + 0.0119363
[440]	cv_agg's res: 0.864686 + 0.0119005
[442]	cv_agg's res: 0.864972 + 0.0113935
[444]	cv_agg's res: 0.864804 + 0.0111735
[446]	cv_agg's res: 0.864531 + 0.0114047
[448]	cv_agg's res: 0.864864 + 0.0115608
[450]	cv_agg's res: 0.864824 + 0.011063
[452]	cv_agg's res: 0.86482 + 0.0111176
[454]	cv_agg's res: 0.86

{'res-mean': [0.6476387590597426,
  0.6499951623295945,
  0.6591715864635503,
  0.6618167740637607,
  0.6625647675360523,
  0.6637660370546464,
  0.6641492020006673,
  0.6674718340909012,
  0.6753761817788378,
  0.6767433248496811,
  0.6763907746913974,
  0.6765127379450124,
  0.6783791760847655,
  0.6795004719466572,
  0.7586961682910713,
  0.7840756556817295,
  0.8028270678393157,
  0.8107516215635439,
  0.8164904662181472,
  0.823754094452004,
  0.8265343123760357,
  0.8309748462410105,
  0.8337353952175537,
  0.8380349093275727,
  0.8386642395002939,
  0.8392923745841285,
  0.840308888310544,
  0.8408948690507166,
  0.8409760877041744,
  0.8402557975704537,
  0.8420949407339419,
  0.8427417275144921,
  0.8442629428172923,
  0.845231157492293,
  0.8448902882501251,
  0.8447121130952091,
  0.8455009085226429,
  0.8474360725592662,
  0.8472861337143964,
  0.8479168469698418,
  0.849299925283615,
  0.849788304858014,
  0.8506817948914546,
  0.8497663180492108,
  0.8504524361766305,
  0

In [16]:
model3 =lgb.train(lgb_params3,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=385,valid_sets=[dtrain])

[100]	training's res: 0.942454
[200]	training's res: 0.978334
[300]	training's res: 0.995345


In [17]:
lgb.cv(lgb_params4,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.650145 + 0.00536924
[4]	cv_agg's res: 0.670275 + 0.00825709
[6]	cv_agg's res: 0.670577 + 0.00820361
[8]	cv_agg's res: 0.674433 + 0.00859562
[10]	cv_agg's res: 0.678263 + 0.00665971
[12]	cv_agg's res: 0.678281 + 0.00650974
[14]	cv_agg's res: 0.679692 + 0.00573394
[16]	cv_agg's res: 0.783997 + 0.00542312
[18]	cv_agg's res: 0.813393 + 0.0129557
[20]	cv_agg's res: 0.825486 + 0.0127947
[22]	cv_agg's res: 0.83152 + 0.0111832
[24]	cv_agg's res: 0.8377 + 0.0122617
[26]	cv_agg's res: 0.839672 + 0.0118317
[28]	cv_agg's res: 0.843035 + 0.0120845
[30]	cv_agg's res: 0.844 + 0.0116558
[32]	cv_agg's res: 0.844425 + 0.0123061
[34]	cv_agg's res: 0.845944 + 0.011238
[36]	cv_agg's res: 0.846611 + 0.0119783
[38]	cv_agg's res: 0.846857 + 0.00999082
[40]	cv_agg's res: 0.848483 + 0.0100756
[42]	cv_agg's res: 0.848173 + 0.0112998
[44]	cv_agg's res: 0.848387 + 0.0115441
[46]	cv_agg's res: 0.848478 + 0.0111348
[48]	cv_agg's res: 0.851104 + 0.0112175
[50]	cv_agg's res: 0.852492 + 0.0106967
[5

[406]	cv_agg's res: 0.867584 + 0.0140497
[408]	cv_agg's res: 0.867903 + 0.0146818
[410]	cv_agg's res: 0.867898 + 0.0143733
[412]	cv_agg's res: 0.86809 + 0.0145892
[414]	cv_agg's res: 0.868209 + 0.0143598
[416]	cv_agg's res: 0.868078 + 0.0148528
[418]	cv_agg's res: 0.868081 + 0.0145343
[420]	cv_agg's res: 0.868346 + 0.014266
[422]	cv_agg's res: 0.868159 + 0.0143317
[424]	cv_agg's res: 0.868028 + 0.0145545
[426]	cv_agg's res: 0.867784 + 0.0141616
[428]	cv_agg's res: 0.868189 + 0.0136838
[430]	cv_agg's res: 0.867597 + 0.0137732
[432]	cv_agg's res: 0.868676 + 0.0142804
[434]	cv_agg's res: 0.868261 + 0.0136544
[436]	cv_agg's res: 0.868201 + 0.0137601
[438]	cv_agg's res: 0.868616 + 0.0141207
[440]	cv_agg's res: 0.869296 + 0.0147395
[442]	cv_agg's res: 0.869465 + 0.0149776
[444]	cv_agg's res: 0.868865 + 0.0144921
[446]	cv_agg's res: 0.868571 + 0.0146068
[448]	cv_agg's res: 0.869067 + 0.01474
[450]	cv_agg's res: 0.868923 + 0.0143855
[452]	cv_agg's res: 0.868179 + 0.0144027
[454]	cv_agg's res: 

{'res-mean': [0.6456114818502771,
  0.6501452266105988,
  0.6553832130603559,
  0.6702749507110307,
  0.6698230311527898,
  0.6705766999540642,
  0.670548729416578,
  0.6744332078300492,
  0.6782167295436762,
  0.6782633612083453,
  0.6783031415283259,
  0.6782809693123575,
  0.678859219341008,
  0.6796915917086128,
  0.7529096297723495,
  0.7839965663273332,
  0.8026881438738723,
  0.8133930220967525,
  0.8203498278511989,
  0.8254861377861955,
  0.8276791673723306,
  0.8315199933785461,
  0.8329083568982942,
  0.8377004263169548,
  0.8381854145262323,
  0.8396718561870635,
  0.83934962969253,
  0.843035261562556,
  0.8439359571251552,
  0.8440002491601634,
  0.8433760928149819,
  0.8444250374267993,
  0.8452577128297968,
  0.8459435383861496,
  0.8466841446054252,
  0.8466106832121908,
  0.8456913853861727,
  0.8468571535925397,
  0.8470431824017238,
  0.8484830111299687,
  0.8492639479445443,
  0.8481733004385861,
  0.8487153239509252,
  0.8483868578501695,
  0.848970321582243,
  0.

In [18]:
model4 =lgb.train(lgb_params4,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=442,valid_sets=[dtrain])

[100]	training's res: 0.941766
[200]	training's res: 0.977112
[300]	training's res: 0.995562
[400]	training's res: 0.999556


In [19]:
lgb.cv(lgb_params5,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.668996 + 0.000597736
[4]	cv_agg's res: 0.673592 + 0.00298789
[6]	cv_agg's res: 0.674595 + 0.00346772
[8]	cv_agg's res: 0.676471 + 0.00607294
[10]	cv_agg's res: 0.68031 + 0.00439556
[12]	cv_agg's res: 0.680312 + 0.00403416
[14]	cv_agg's res: 0.685251 + 0.00388601
[16]	cv_agg's res: 0.787903 + 0.00492584
[18]	cv_agg's res: 0.811161 + 0.0095677
[20]	cv_agg's res: 0.826134 + 0.0106785
[22]	cv_agg's res: 0.833594 + 0.00977541
[24]	cv_agg's res: 0.837517 + 0.0120293
[26]	cv_agg's res: 0.840966 + 0.0120873
[28]	cv_agg's res: 0.843622 + 0.0130275
[30]	cv_agg's res: 0.846651 + 0.0138245
[32]	cv_agg's res: 0.845763 + 0.0134753
[34]	cv_agg's res: 0.847567 + 0.013917
[36]	cv_agg's res: 0.848997 + 0.0135576
[38]	cv_agg's res: 0.849414 + 0.0124928
[40]	cv_agg's res: 0.850379 + 0.0126931
[42]	cv_agg's res: 0.852235 + 0.0120108
[44]	cv_agg's res: 0.852399 + 0.0108025
[46]	cv_agg's res: 0.852968 + 0.0100614
[48]	cv_agg's res: 0.852835 + 0.00979916
[50]	cv_agg's res: 0.853796 + 0.010

{'res-mean': [0.6563872048732301,
  0.6689957328435892,
  0.6693907984330624,
  0.6735919860161359,
  0.6735981924086775,
  0.6745950212005695,
  0.6743809674201645,
  0.6764710857821824,
  0.6799934302591035,
  0.6803102576015045,
  0.6803952763252882,
  0.6803123390136373,
  0.6808991335425579,
  0.685250648205835,
  0.7585521763474583,
  0.7879025825664009,
  0.8043550338147978,
  0.8111608631144854,
  0.8213630739926016,
  0.8261344442898991,
  0.8294893819880107,
  0.8335940314164588,
  0.8370532996852872,
  0.8375170947645851,
  0.8401253847859002,
  0.8409657462456108,
  0.8424896299678238,
  0.843621809959746,
  0.8442703788271623,
  0.8466512393514488,
  0.845464603823478,
  0.8457628980781177,
  0.8466045143912483,
  0.8475668020025068,
  0.8482261296376333,
  0.8489968754518958,
  0.8490696924473609,
  0.8494136679863263,
  0.8492890340880738,
  0.8503791964901186,
  0.8514312448543301,
  0.8522350230117427,
  0.8518643400898358,
  0.8523989644728616,
  0.8532267313210827,
 

In [20]:
model5 =lgb.train(lgb_params5,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=302,valid_sets=[dtrain])

[100]	training's res: 0.944729
[200]	training's res: 0.977678
[300]	training's res: 0.994478


In [21]:
lgb.cv(lgb_params6,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.668673 + 0.00607203
[4]	cv_agg's res: 0.676113 + 0.00196075
[6]	cv_agg's res: 0.67782 + 0.00233219
[8]	cv_agg's res: 0.678656 + 0.00260095
[10]	cv_agg's res: 0.678621 + 0.00264273
[12]	cv_agg's res: 0.681962 + 0.00529363
[14]	cv_agg's res: 0.681948 + 0.00542533
[16]	cv_agg's res: 0.787083 + 0.00653674
[18]	cv_agg's res: 0.81586 + 0.00971707
[20]	cv_agg's res: 0.827157 + 0.01213
[22]	cv_agg's res: 0.833658 + 0.0127875
[24]	cv_agg's res: 0.838039 + 0.0143394
[26]	cv_agg's res: 0.843137 + 0.0167788
[28]	cv_agg's res: 0.845141 + 0.0128275
[30]	cv_agg's res: 0.847609 + 0.0141959
[32]	cv_agg's res: 0.848989 + 0.0128743
[34]	cv_agg's res: 0.851224 + 0.0112681
[36]	cv_agg's res: 0.851663 + 0.011012
[38]	cv_agg's res: 0.852136 + 0.0109401
[40]	cv_agg's res: 0.853369 + 0.0110716
[42]	cv_agg's res: 0.852662 + 0.0110363
[44]	cv_agg's res: 0.852964 + 0.0117756
[46]	cv_agg's res: 0.85252 + 0.0125252
[48]	cv_agg's res: 0.852504 + 0.0117107
[50]	cv_agg's res: 0.852908 + 0.0115854
[

{'res-mean': [0.644624389997167,
  0.6686731153910733,
  0.6711746328940356,
  0.6761133638681258,
  0.6767701980938244,
  0.67781987172626,
  0.6782710927797259,
  0.678656041385269,
  0.6782770010626393,
  0.6786209021145201,
  0.6812712402279609,
  0.6819618338516994,
  0.681607261546191,
  0.6819476425838481,
  0.757279058670605,
  0.7870829371966005,
  0.8044996968916106,
  0.8158602230344448,
  0.8224288300698722,
  0.8271565705406768,
  0.8320248725165508,
  0.8336584967895542,
  0.8358432950316551,
  0.8380388809895306,
  0.840035042675046,
  0.8431372161125871,
  0.8434756118544581,
  0.8451413404327877,
  0.8463993686882892,
  0.8476092321319303,
  0.8470704744649514,
  0.8489888774014916,
  0.8500171716225028,
  0.8512240825375207,
  0.851356916265694,
  0.8516628830360963,
  0.8521090058688018,
  0.8521362431392928,
  0.8524016071499029,
  0.8533685666459657,
  0.8527357385593626,
  0.85266193151972,
  0.8522519990540633,
  0.8529635511464017,
  0.8518010436091735,
  0.8525

In [22]:
model6 =lgb.train(lgb_params6,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=190,valid_sets=[dtrain])

[100]	training's res: 0.942566


In [23]:
lgb.cv(lgb_params7,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.652572 + 0.00511888
[4]	cv_agg's res: 0.66053 + 0.00104812
[6]	cv_agg's res: 0.674099 + 0.0024253
[8]	cv_agg's res: 0.677425 + 0.00422416
[10]	cv_agg's res: 0.679262 + 0.00494166
[12]	cv_agg's res: 0.679355 + 0.00532317
[14]	cv_agg's res: 0.681355 + 0.00544808
[16]	cv_agg's res: 0.780176 + 0.00713211
[18]	cv_agg's res: 0.811605 + 0.0112081
[20]	cv_agg's res: 0.827306 + 0.0111416
[22]	cv_agg's res: 0.834666 + 0.0115393
[24]	cv_agg's res: 0.837835 + 0.0109608
[26]	cv_agg's res: 0.841331 + 0.0135667
[28]	cv_agg's res: 0.843774 + 0.0133228
[30]	cv_agg's res: 0.847277 + 0.0124148
[32]	cv_agg's res: 0.848494 + 0.0123406
[34]	cv_agg's res: 0.848349 + 0.0117333
[36]	cv_agg's res: 0.848666 + 0.0104907
[38]	cv_agg's res: 0.850287 + 0.0112768
[40]	cv_agg's res: 0.850738 + 0.0107393
[42]	cv_agg's res: 0.851757 + 0.0106903
[44]	cv_agg's res: 0.851685 + 0.011204
[46]	cv_agg's res: 0.852194 + 0.010999
[48]	cv_agg's res: 0.851713 + 0.0116646
[50]	cv_agg's res: 0.853094 + 0.0101109


{'res-mean': [0.6484102658843108,
  0.6525719353767712,
  0.6601590430100557,
  0.6605303096332377,
  0.6741827355851266,
  0.6740988825235408,
  0.6763691224714448,
  0.6774247042082849,
  0.6788703259774995,
  0.6792622035790417,
  0.6792769139503649,
  0.6793550168877979,
  0.6808883899928828,
  0.6813554922566237,
  0.7528868244260322,
  0.7801764963616405,
  0.7989798309108792,
  0.8116050572307317,
  0.8208503729241331,
  0.8273060103570655,
  0.8308635828473125,
  0.8346660736124526,
  0.835854291298912,
  0.8378353959838992,
  0.8399725539262685,
  0.8413314453848321,
  0.8406928452360135,
  0.8437742388734822,
  0.8450581237208441,
  0.8472769987660884,
  0.8465611304851887,
  0.8484939649806958,
  0.8473956344706802,
  0.8483488383957002,
  0.8487789236275355,
  0.8486664489188174,
  0.8489969456046124,
  0.8502872252213097,
  0.8504398920830063,
  0.8507381317637274,
  0.8509120198473658,
  0.851756762052358,
  0.8508416685692999,
  0.85168543288369,
  0.8516337873922826,
  

In [24]:
model7 =lgb.train(lgb_params7,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=236,valid_sets=[dtrain])

[100]	training's res: 0.94142
[200]	training's res: 0.978131


In [25]:
lgb.cv(lgb_params8,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.657136 + 0.00373071
[4]	cv_agg's res: 0.664227 + 0.000509567
[6]	cv_agg's res: 0.669185 + 0.00373428
[8]	cv_agg's res: 0.670167 + 0.00507123
[10]	cv_agg's res: 0.67036 + 0.00467598
[12]	cv_agg's res: 0.677352 + 0.00358097
[14]	cv_agg's res: 0.679734 + 0.0055585
[16]	cv_agg's res: 0.784956 + 0.00497014
[18]	cv_agg's res: 0.813546 + 0.0106192
[20]	cv_agg's res: 0.824569 + 0.0138723
[22]	cv_agg's res: 0.832548 + 0.0165033
[24]	cv_agg's res: 0.837737 + 0.0151734
[26]	cv_agg's res: 0.839527 + 0.0145807
[28]	cv_agg's res: 0.841596 + 0.0138052
[30]	cv_agg's res: 0.844302 + 0.0138208
[32]	cv_agg's res: 0.844475 + 0.0144624
[34]	cv_agg's res: 0.844629 + 0.0116726
[36]	cv_agg's res: 0.846509 + 0.0117641
[38]	cv_agg's res: 0.846783 + 0.0121453
[40]	cv_agg's res: 0.848001 + 0.0104704
[42]	cv_agg's res: 0.850347 + 0.0112342
[44]	cv_agg's res: 0.850203 + 0.0110565
[46]	cv_agg's res: 0.850636 + 0.0107302
[48]	cv_agg's res: 0.850235 + 0.0106487
[50]	cv_agg's res: 0.850577 + 0.00981

{'res-mean': [0.6466916811945788,
  0.6571362315542203,
  0.6602874516794407,
  0.6642268911547963,
  0.6692947012771713,
  0.6691847536266499,
  0.6686076758650211,
  0.6701672467371043,
  0.6701610208871082,
  0.6703596563304508,
  0.6751611158537679,
  0.6773517189384708,
  0.6785791390977506,
  0.67973383851232,
  0.7564498668144553,
  0.7849561631298125,
  0.8036451269284988,
  0.8135457307716023,
  0.8202250179586773,
  0.8245690149034169,
  0.8281452328265612,
  0.8325475914957717,
  0.8346258336456941,
  0.837737198181474,
  0.8379922570557619,
  0.8395268712982551,
  0.8414845087215038,
  0.8415956122828497,
  0.8428620731987296,
  0.8443019009355122,
  0.8440008920493787,
  0.8444752636585994,
  0.8445610968822427,
  0.844629217948308,
  0.845241206299615,
  0.8465091877315943,
  0.8473861015596972,
  0.8467825174282019,
  0.8472171826098384,
  0.848001492153125,
  0.8494114201202129,
  0.8503472535157663,
  0.8494964720343856,
  0.85020305770232,
  0.8502648922593655,
  0.85

In [26]:
model8 =lgb.train(lgb_params8,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=216,valid_sets=[dtrain])

[100]	training's res: 0.94159
[200]	training's res: 0.976441


In [27]:
lgb.cv(lgb_params9,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.666622 + 0.00388042
[4]	cv_agg's res: 0.676609 + 0.00318441
[6]	cv_agg's res: 0.677912 + 0.00342598
[8]	cv_agg's res: 0.679623 + 0.00202348
[10]	cv_agg's res: 0.679051 + 0.00208213
[12]	cv_agg's res: 0.681057 + 0.00312535
[14]	cv_agg's res: 0.688701 + 0.00940727
[16]	cv_agg's res: 0.782917 + 0.00454304
[18]	cv_agg's res: 0.816516 + 0.00968997
[20]	cv_agg's res: 0.828564 + 0.0117747
[22]	cv_agg's res: 0.834356 + 0.0110789
[24]	cv_agg's res: 0.837916 + 0.010688
[26]	cv_agg's res: 0.840864 + 0.0115842
[28]	cv_agg's res: 0.844304 + 0.0106256
[30]	cv_agg's res: 0.843745 + 0.010464
[32]	cv_agg's res: 0.847023 + 0.01095
[34]	cv_agg's res: 0.849664 + 0.0117505
[36]	cv_agg's res: 0.85075 + 0.0112257
[38]	cv_agg's res: 0.850055 + 0.0122526
[40]	cv_agg's res: 0.850554 + 0.0113631
[42]	cv_agg's res: 0.851093 + 0.0103515
[44]	cv_agg's res: 0.85173 + 0.0107103
[46]	cv_agg's res: 0.851933 + 0.0100708
[48]	cv_agg's res: 0.852676 + 0.0099608
[50]	cv_agg's res: 0.852583 + 0.0100615
[

[406]	cv_agg's res: 0.867413 + 0.0131342
[408]	cv_agg's res: 0.866989 + 0.012501
[410]	cv_agg's res: 0.867607 + 0.0123075
[412]	cv_agg's res: 0.86724 + 0.0122223
[414]	cv_agg's res: 0.867286 + 0.0121202
[416]	cv_agg's res: 0.867699 + 0.012002
[418]	cv_agg's res: 0.867431 + 0.0116033
[420]	cv_agg's res: 0.867904 + 0.0118619
[422]	cv_agg's res: 0.867784 + 0.0119645
[424]	cv_agg's res: 0.867509 + 0.0119473
[426]	cv_agg's res: 0.867354 + 0.0117145
[428]	cv_agg's res: 0.867849 + 0.0121116
[430]	cv_agg's res: 0.867702 + 0.0121669
[432]	cv_agg's res: 0.867846 + 0.0120539
[434]	cv_agg's res: 0.86757 + 0.0122776
[436]	cv_agg's res: 0.867566 + 0.0122713
[438]	cv_agg's res: 0.867581 + 0.0122882
[440]	cv_agg's res: 0.867879 + 0.0121152
[442]	cv_agg's res: 0.86772 + 0.0122023
[444]	cv_agg's res: 0.867898 + 0.0121356
[446]	cv_agg's res: 0.867458 + 0.0123558
[448]	cv_agg's res: 0.867658 + 0.0126052
[450]	cv_agg's res: 0.86762 + 0.0126805
[452]	cv_agg's res: 0.867901 + 0.012413
[454]	cv_agg's res: 0.8

{'res-mean': [0.6415666895398633,
  0.6666216718442899,
  0.669841631944012,
  0.6766085502009239,
  0.6773406887037282,
  0.6779118769258631,
  0.6796620014357226,
  0.6796234926336343,
  0.678984450911496,
  0.6790511101879,
  0.6800634992313178,
  0.6810574408225379,
  0.681508652772057,
  0.6887009403100525,
  0.7544683143700004,
  0.7829165855044181,
  0.8056885933740253,
  0.8165162039569308,
  0.8243623179414418,
  0.82856404868112,
  0.8304042589424109,
  0.8343557086898482,
  0.8351992485253023,
  0.8379156709818054,
  0.8387848488961165,
  0.8408636140446885,
  0.841384258208764,
  0.8443038633039578,
  0.842699928846378,
  0.8437454814685582,
  0.8457824988430671,
  0.8470225236076733,
  0.8494877919689489,
  0.8496637804010904,
  0.8512983662313959,
  0.8507497705034525,
  0.8498004210676106,
  0.8500552017535664,
  0.85063565495858,
  0.8505542787644881,
  0.8523458379018032,
  0.8510929026825694,
  0.8515278996748067,
  0.8517301510626846,
  0.8513928946493919,
  0.851933

In [28]:
model9 =lgb.train(lgb_params9,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=490,valid_sets=[dtrain])

[100]	training's res: 0.941368
[200]	training's res: 0.977084
[300]	training's res: 0.994253
[400]	training's res: 0.999778


### 预测

In [29]:
pred0=model.predict(test.drop(['uid'],axis=1))

In [30]:
pred1=model1.predict(test.drop(['uid'],axis=1))

In [31]:
pred2=model2.predict(test.drop(['uid'],axis=1))

In [32]:
pred3=model3.predict(test.drop(['uid'],axis=1))

In [33]:
pred4=model4.predict(test.drop(['uid'],axis=1))

In [34]:
pred5=model5.predict(test.drop(['uid'],axis=1))

In [35]:
pred6=model6.predict(test.drop(['uid'],axis=1))

In [36]:
pred7=model7.predict(test.drop(['uid'],axis=1))

In [37]:
pred8=model8.predict(test.drop(['uid'],axis=1))

In [38]:
pred9=model9.predict(test.drop(['uid'],axis=1))

In [39]:
pred =pd.DataFrame({'p0':pred0,'p1':pred1,'p2':pred2,'p3':pred3,'p4':pred4,'p5':pred5,'p6':pred6,'p7':pred7,'p8':pred8,'p9':pred9})
pred.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9
0,0.503611,0.483499,0.629334,0.683364,0.566530,0.532328,0.527107,0.495057,0.622986,0.709259
1,0.667142,0.689414,0.707339,0.724159,0.796947,0.703698,0.739236,0.646715,0.677277,0.767419
2,0.030067,0.042621,0.025439,0.017178,0.009782,0.026628,0.042149,0.035172,0.043109,0.009917
3,0.043580,0.033085,0.029403,0.021574,0.015629,0.027175,0.048683,0.036951,0.045819,0.014774
4,0.094979,0.098898,0.068565,0.054892,0.047479,0.066062,0.098660,0.104815,0.114316,0.040317


In [40]:
# 求平均值
pred_t = pred.T
pred_mean = pred_t.mean()
res =pd.DataFrame({'uid':test.uid,'label':pred_mean})

In [41]:
res=res.sort_values(by='label',ascending=False)
res.label=res.label.map(lambda x: 1 if x>=0.38 else 0)
#res.label = res.label.map(lambda x: int(x))

In [42]:
res.to_csv('../result/lgb-baseline.csv',index=False,header=False,sep=',',columns=['uid','label'])